In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Hadoop and MapReduce
<!-- requirement: small_data/gutenberg -->
<!-- requirement: projects/MR -->


So far, we have only done calculations for which all the memory required can fit into ram.  With Big Data, we often run into situations where we have to process data that exceeds practical ram limitations.


### (Anti-)Example

The most expensive laptop that Apple will sell you has 16 GB of ram (and it's over \$2,000).  The biggest instance that Digital Ocean will rent you has at most 64 GB of ram (and it's \$640 / month).  The IRS has tax records for ~300M Americans.  If each record has over 54 Bytes, it will not fit into the RAM of any Apple laptop.  If each record has more than 214 Bytes, it will not fit onto any Digital Ocean instance.  Remember that IRS form 1099EZ on has 30-or-so numbers to fill in, each of which (stored as an integer) is 4 bytes, so each record is at least 120 Bytes.  And that doesn't include any supplemental forms ...

We also run into computational bottlenecks, where doing all the computations on one CPU is prohibitively slow.  The answer is to distribute the computation onto multiple computers and to be able to take advantage of their collective CPU and memory.  You might already be aware of some **parallel computation** platforms.  We're going to talk about Hadoop and MapReduce, which is a very powerful paradigm for computation.

## What is Hadoop?

[Hadoop](http://hadoop.apache.org/) is a low-level system for distributed computation on a cluster.  It can also refer to the broader "Hadoop project" ecosystem that adds on extra functionality on top of this.  The two base layers of functionality are:


## HDFS

A distributed file system.  When you want to perform a computation on Hadoop, the files involved live "on" HDFS: In reality, they are spread out in chunks, potentially with some replication, across the various machines in your cluster.  HDFS is the system responsible for this chunking, replicating, moving, etc.


## MapReduce and YARN

A system for actually distributing computations across the cluster.  Traditionally this was the `MapReduce` framework (details later).  In Hadoop 2.0, this layer is called `YARN` and has been split up further: There's a general half that can be used for all types of workloads (not just MapReduce), and a specific half that does MapReduce.


## Things that Hadoop is not

 - The only game in town for distributed computation.
 - Good for running (most) machine learning algorithms (.. it can however be good for pre-processing / feature generation).
 - Good for real-time (as opposed to batch) processing.


## Hadoop ecosystem technologies


The default way to run a computation on Hadoop is to write a MapReduce, either from scratch or using some intermediate framework (e.g. `mrjob`).  

There are also several projects that allow one to use Hadoop without actively worrying about MapReduce.  The two most notable ones:

- [Hive](https://hive.apache.org/): This lets you run SQL-like commands on top of Hadoop/MR.  The language is called `HiveQL` and is a dialect of SQL with some quirks and special features.
- [Pig](http://pig.apache.org/): A special purpose language for certain simple common operations.

## Useful HDFS commands


**Hadoop** is a distributed file system for storing data on a *distributed disk* (a collective hard disks of multiple computers in a cluster).  Hadoop commands resemble Unix commands so you should be familiar with them already.

Let's get started with Hadoop.  It's already installed on your JupyterHub pod.  First, here's some stuff to test out Hadoop and MapReduce.  If these commands fail, read the notes on restarting Hadoop at the end of this document.

``` bash

# create folder on hadoop

hadoop fs -mkdir input


# move these files to hadoop

hadoop fs -put /etc/hadoop/conf/*.xml input


# check that those files are on hadoop

hadoop fs -ls input


# run this mapreduce

hadoop jar /usr/lib/hadoop-mapreduce/hadoop-mapreduce-examples-2.2.0.2.0.11.0-1.jar grep input output '.*yarn.*'


# note that there is an output dir

hadoop fs -ls


# it contains a _SUCCESS flag and part-r-00000

hadoop fs -ls output


# here's the contents

hadoop fs -cat output/part-r-00000 | head
```
 
Notice that the above code copies some of your Hadoop configuration files `/etc/hadoop/conf/*.xml` onto HDFS (Hadoop Distributed File System) under the folder `input`.  It then runs a `grep` MapReduce looking for words that start with `dfs` and places the results into the folder `output`.  You should verify its contents look something like this:

```
1	      <name>yarn.scheduler.capacity.root.queues</name>
1	      <name>yarn.scheduler.capacity.root.default.user-limit-factor</name>
1	      <name>yarn.scheduler.capacity.root.default.state</name>
1	      <name>yarn.scheduler.capacity.root.default.maximum-capacity</name>
1	      <name>yarn.scheduler.capacity.root.default.capacity</name>
1	      <name>yarn.scheduler.capacity.root.default.acl_submit_applications</name>
1	      <name>yarn.scheduler.capacity.root.default.acl_administer_queue</name>
1	      <name>yarn.scheduler.capacity.node-locality-delay</name>
1	      <name>yarn.scheduler.capacity.maximum-applications</name>
1	      <name>yarn.scheduler.capacity.maximum-am-resource-percent</name>
1	      <name>yarn.resourcemanager.hostname</name>
```

If these commands do not work due to a "connection error", you may need to restart Hadoop:

```bash
sudo su root  # become the root user
echo JAVA_HOME=/usr/lib/jvm/java-6-openjdk-amd64 >> /etc/environment

# run command-D to get out of root mode


# Start HDFS

sudo service hadoop-hdfs-datanode start
sudo service hadoop-hdfs-namenode init
sudo service hadoop-hdfs-namenode start
```

## Building a MapReduce app


MapReduce is written in Java which means that directory tree is deep and the code is loquacious.  Open up [`WordCount.java`](projects/MR/src/main/java/com/thedataincubator/hadoopexamples/WordCount.java).  Yes - word count is going to be 100 lines long.  Let's begin with the mapper, which declares that is of type `Mapper <k0, v0, k1, v1>`:

``` java
public static class WordTokenizerMapper
  extends Mapper<Object, Text, Text, IntWritable>
```
      
The meat of the code is these 5 lines:

``` java
IntWritable one = new IntWritable(1);
Text word = new Text();

StringTokenizer tokenizer 
    = new StringTokenizer(value.toString(), " \t\n\r\f,.:;?![]'\"");
String s = tokenizer.nextToken().toLowerCase().trim();
word.set(s);
context.write(word, one);
```

Because MapReduce has to write everything to disk, it needs to use wrappers (called `Writable`s) for basic types which can *serialize* basic data types.  `IntWritable` is a wrapper for `int` and `Text` is a wrapper for `String`.  `IntWritable` has a `set` method which sets the value being wrapped.  The string is split by `StringTokenizer` into words and then each word is outputted with the number `one` (wrapped as an `IntWritable`).

The reducer has to declare that it is of type `Reducer <k1, v1, k2, v2>` (and note that `k1` and `v1` in the mapper and reducer match):

``` java
public static class WordOccurrenceReducer
  extends Reducer<Text, IntWritable, Text, IntWritable>
```
      
Its logic is expressed (rather garrulously) in a few lines:
      
``` java
private IntWritable occurrencesOfWord = new IntWritable();

int sum = 0;
for (IntWritable val : values) {
    sum += val.get();
}
occurrencesOfWord.set(sum);
context.write(key, occurrencesOfWord);
```
    
Finally, there's a `main` function which is the entry-point of a Java program and where the mapper, `FileInputFormat`, `FileOutputFormat` etc ... are specified.  You'll notice that Java is a little slow at learning and you'll have to specify the same things in multiple places.

``` java
public static void main(String[] args)
  throws Exception
```
      
**Exercise:**
1. Add a `combiner` to speed things up.  You can read about the `combiner` in the [Apache Tutorial](https://hadoop.apache.org/docs/r1.2.1/mapred_tutorial.html) (just look for the word combiner).
1. You should have all the basic ingredients to write a Hadoop version of Unix's word count.  That is, accept a file and write 3 outputs, which correspond to the number of words, the number of characters in those words, and the number of lines.
1. [`SpendingByUser.java`](projects/MR/src/main/java/com/thedataincubator/hadoopexamples/SpendingByUser.java) is a very similar program that reads in a tab-delineated file [`transactions.tsv`](projects/MR/data/transactions.tsv) representing transactions at a hypothetical store.  The MapReduce outputs a pair of `userid` and total spending amount, e.g. :
    
        u1, 150
    
 Modify it to output the average purchase amount and standard deviation of the purchase amounts:
  
        u1 mean, 50
        u1 std, 23
  
 How would you use a combiner in this case?

### A note about how to run MapReduce


Download some sample texts from Project Gutenberg and move them to HDFS.  We are going to use them to count our data file.

``` bash

# copy data and check that it made it

cd ~/datacourse
hadoop fs -copyFromLocal small_data/gutenberg gutenberg
hadoop fs -ls gutenberg
```
    
This example is based on [this tutorial](http://www.michael-noll.com/tutorials/running-hadoop-on-ubuntu-linux-single-node-cluster/).  First, you should make sure you can find the [`pom.xml`](projects/MR/pom.xml).  This is an `xml` file that serves as a `makefile` for `maven`, a Java compile tool.  `cd` to the containing folder on your pod and invoke the maven compile command

``` bash
 cd projects/MR/
 mvn clean install
```

If this is your first time running things, maven will download a lot of stuff (this could take some time).  Eventually, you should see a build success message like this
```
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total time: 2.288 s
[INFO] Finished at: 2014-05-25T19:40:08-05:00
[INFO] Final Memory: 16M/81M
[INFO] ------------------------------------------------------------------------
```
If instead of `BUILD SUCCESS` you see `BUILD FAILURE`, look immediately above it for the build errors, fix them, and try again.  When you see `BUILD SUCCESS`, you should find `hadoop-examples-1.0.jar` under `target`:

``` bash
ls target/hadoop-examples-1.0.jar
```

Confirm that the file is there.  Once it is, you can run 

``` bash

### run mapreduce

hadoop jar target/hadoop-examples-1.0.jar com.thedataincubator.hadoopexamples.WordCount gutenberg gutenberg-output


### retrieve the data

hadoop fs -cat gutenberg-output/part-r-00000 | more
```
 
You should see something like 
```
14/05/25 19:56:29 INFO input.FileInputFormat: Total input paths to process : 3
14/05/25 19:56:30 INFO mapred.JobClient: Running job: job_201405231953_0030
14/05/25 19:56:31 INFO mapred.JobClient:  map 0% reduce 0%
14/05/25 19:56:42 INFO mapred.JobClient:  map 67% reduce 0%
14/05/25 19:56:48 INFO mapred.JobClient:  map 100% reduce 0%
14/05/25 19:56:51 INFO mapred.JobClient:  map 100% reduce 100%
14/05/25 19:56:53 INFO mapred.JobClient: Job complete: job_201405231953_0030
```
Followed by a bunch of stats about Hadoop's performance.  This output is periodically give you updates on the progress of the map and reduce phase.  *NB*: `com.thedataincubator.hadoopexamples.WordCount` corresponds to two places:

1. The first line in `WordCount.java` is `package com.thedataincubator.hadoopexamples;`  This is necessary for invoking the command line when running Hadoop: it tells the JVM where to find the package at runtime.
1. The file is located in `src/main/java/com/thedataincubator/hadoopexamples/WordCount.java`.  This is done for Maven: it expects to find the files there when it's looking for files to compile.

As you add and modify files for this lesson, keep these two invariants constant.  Otherwise, you'll spend your time fighting either Maven or the JVM.

## Joining data using MapReduce


We are running a store and we have information about our users in [`users.tsv`](projects/MR/data/users.tsv) by user id, email, language, country:

    u1	bob@exmaple.com	EN	US
    u2	jon@exmaple.com	EN	GB
    u3	sally@exmaple.com	FR	FR
    u4	katie@exmaple.com	FR	FR
    
We also have their transactions in [`transactions.tsv`](projects/MR/data/transactions.tsv) by transaction id, product id, user id, and amount:
    
    t1	p1	u1	100
    t2	p4	u2	150
    t3	p3	u1	200
    t4	p4	u4	50
    t5	p1	u3	100
    t6	p2	u1	100
    
What if we wanted to join these two data sets together by `userid` to get the amount spent in each country?  To do this in MapReduce, we have to first join the data.

Joining data in MapReduce requires that we use two mappers, one for `users.tsv` and the other for `transactions.tsv`.  The trick is that the output of both mappers must be the same type (i.e. `(k1, v1)` have to be the same for both mappers) so that they can be fed to a common reducer which performs the join.  What types will we use?  The key `k1` is clear: we're joining these two tables on user id.  The value is less clear.  We want `users.tsv` to output the country of the user, but we want `transactions.tsv` to output the amount that user spent on each transaction.  We can do both by creating a special `Writable` that is the union of the two, which is done in [`AmountOrCountry.java`](projects/MR/src/main/java/com/thedataincubator/hadoopexamples/AmountOrCountry.java):

```java
public class AmountOrCountry
implements Writable
{
  public AmountOrCountry(){}

  public Text type = new Text();
  public IntWritable amount = new IntWritable();
  public Text country = new Text();

  public void setAmount(int amount) {
    type.set("amount");
    this.amount = new IntWritable(amount);
  }

  public void setCountry(String country) {
    type.set("country");
    this.country = new Text(country);
  }

  public void write(DataOutput out) throws IOException {
    type.write(out);
    amount.write(out);
    country.write(out);
  }

  public void readFields(DataInput in) throws IOException {
    type.readFields(in);
    amount.readFields(in);
    country.readFields(in);
  }
}
```

The class has 3 fields: `type`, `amount`, and `country`.  If `type == "amount"`, then the `amount` field is used and the `country` field is ignored, and vice-versa if `type == "country"`.  All `Writable` classes have to implement `write` and `readFields` which write the data out to disk and read them back in.  They do this by calling the methods on the corresponding values `type`, `amount`, and `country`.  The only trick is to make sure we write the fields out in the same order in which we read them back in.

Then we can see that the output type (`k1, v1`) of `CountryMapper`

```java
public static class CountryMapper
  extends Mapper<LongWritable, Text, Text, AmountOrCountry>
```

matches that of `AmountMapper`

```java
public static class AmountMapper
  extends Mapper<LongWritable, Text, Text, AmountOrCountry>
```

Finally, notice that in `CountryMapper` we use `setCountry` method

```java
public static AmountOrCountry outValue = new AmountOrCountry();

outValue.setCountry(country);
context.write(outKey, outValue);
```

while in `AmountMapper` we use the `setAmount` method

```java
public static AmountOrCountry outValue = new AmountOrCountry();

outValue.setAmount(amount);
context.write(outKey, outValue);
```

The reducer is a little trickier.  We keep track of the running sum (as with `WordCount`) but for each record, we have to check whether we should use the `country` field or the `amount` field:

```java
int sum = 0;
for (AmountOrCountry value: values) {
    if (value.type.toString().equals("amount")) {
        sum += value.amount.get();
    } else { // "country"
        country.set(value.country.toString());
    }
}
sumWritable.set(sum);
context.write(country, sumWritable);
```

Without using the Secondary Sort (for a great explanation, checkout [Matthew Rathbone's blog entry](http://blog.matthewrathbone.com/2013/02/09/real-world-hadoop-implementing-a-left-outer-join-in-hadoop-map-reduce.html)), there's no way to guarantee the order of the records passed to reduce.  The code is written so that the "country" entry could happen at any point.

**Question**: what happens if multiple country values are passed for a user entry?  More likely, what happens if none is?  What does this mean about the user database?

**Exercises**:
1. Let's test it out.  Move the `users.tsv` and `transactions.tsv` files onto HDFS.  Checkout the `main` function to see how to pass parameters to the Hadoop job and test it out!
1. If you look at the output, you'll see we aren't quite done.  There are still multiple entries per country.  We still need to aggregate the results by country.  Write a second MapReduce that aggregates by country (hint, use an identity mapper).
1. For a very comprehensive list of questions and answers, check out [this site](http://meri-stuff.blogspot.com/2011/10/mapreduce-questions-and-answers.html).

## Alternatives to Java MapReduce


As you can see, Java MapReduce turns five lines of SQL into hundreds of lines of Java code.  There are a few well-regarded solutions that try to get around this:
1. **Hadoop Streaming**: This creates a MapReduce by specifying a map and reduce script.  Map and reduce scripts take are take input via `stdin` and output data via `stdout`.  The scripts can be anything (bash, Python, etc ...).  The key and value are separated by the first tab value.  While its very quick to get setup and easy to learn, you have to do a lot of janky serialization.  It also doesn't have real support for joins (instead of having two mappers, one for each input, you need to have one mapper that handles both inputs).  ([Read more](https://hadoop.apache.org/docs/r1.2.1/streaming.html)).  Yelp! built [mrjob](https://github.com/Yelp/mrjob) as a Python wrapper on top of Hadoop streaming.
1. **Hive**: Hive is a SQL interface to MapReduce.  The user writes SQL-like queries and they are translated into MapReduce under the hood.  It only supports a subset of the SQL language.  ([Read more](https://cwiki.apache.org/confluence/display/Hive/Tutorial)).
1. **Pig**: Similar to Hive in that one writes in a relatively declarative language (called Pig Latin) but is more imperative than Hive.  ([Read more](https://pig.apache.org/docs/latest/basic.html)).
1. **Scalding**: Scalding is a highly declarative Scala DSL (domain specific language - think of DSLs as really fancy libraries with their own custom language / syntax).  You treat data as if they were lists in Scala.  For example, here's word count:

```scala
package com.twitter.scalding.examples

import com.twitter.scalding._

class WordCountJob(args : Args) extends Job(args) {
  TextLine( args("input") )
    .flatMap('line -> 'word) { line : String => tokenize(line) }
    .groupBy('word) { _.size }
    .write( Tsv( args("output") ) )

  // Split a piece of text into individual words.
  def tokenize(text : String) : Array[String] = {
    // Lowercase each word and remove punctuation.
    text.toLowerCase.replaceAll("[^a-zA-Z0-9\\s]", "").split("\\s+")
  }
}
```

It's well documented and has a lot of support since it was developed by Twitter.  It's easy to get started with ([Read more](https://github.com/twitter/scalding)).  `Scoobi` is very similar but much less well maintained ([Read more](https://github.com/NICTA/scoobi)).  Spark is similar and developed at Berkeley and promises to cache data processing into ram ([Read more](https://spark.apache.org/))

## Hive

*How does Hive relate to Hadoop?*

Hive is a data warehousing framework built on top of Hadoop. It is meant to allow SQL-like data analysis of large data sets stored on HDFS, S3, or other distributed file systems.

*How does Hive relate to SQL?*

The `HiveQL` querying language is based on SQL, but it does not strictly follow SQL standards and not all commands may be available/equivalent.

*How does Hive relate to MapReduce?*

Just like MapReduce, Hive is best suited for large, relatively uncomplicated batch operations on massive amounts of data. Common applications include log processing, indexing, text mining, etc. From the home page: "Hive is not designed for `OLTP` workloads workloads and does not offer real-time queries or row-level updates. It is best used for batch jobs over large sets of append-only data (like web logs). What Hive values most are scalability...extensibility...fault-tolerance, and loose-coupling with its input formats."

*What does Hive do?*

Hive sessions are managed by a driver app. `HiveQL` queries are parsed and compiled with an optimization engine and translated into a directed acyclic graph (DAG) able to be executed by Hadoop MapReduce, Spark, or other distributed computation engines. A meta-store keeps track of the table namespace, schemas, and partition data.

### Note on restarting Hadoop

If your Hadoop commands do not work, try running these commands

```bash

# Start HDFS

sudo service hadoop-hdfs-datanode start
sudo service hadoop-hdfs-namenode init
sudo service hadoop-hdfs-namenode start
```

### Exit Tickets

1. Describe the difference between declarative and imperative programming languages and how that choice influences MapReduce tasks.

*Copyright &copy; 2018 The Data Incubator.  All rights reserved.*